<a href="https://colab.research.google.com/github/ekanshtrivedi/machine-learning/blob/master/ML_J066_EXP_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
!wget -O AirQualityUCI.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip
!unzip AirQualityUCI.zip
df = pd.read_excel('/content/AirQualityUCI.xlsx', parse_dates=[['Date', 'Time']])
df.replace(to_replace=-200.0, value=0.0, inplace=True)
wv = df['NMHC(GT)']
bad_wv = wv == -200.0
wv[bad_wv] = 0.0

--2020-10-30 16:20:51--  https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1543989 (1.5M) [application/x-httpd-php]
Saving to: ‘AirQualityUCI.zip’

AirQualityUCI.zip   100%[===================>]   1.47M  1.63MB/s    in 0.9s    

2020-10-30 16:20:53 (1.63 MB/s) - ‘AirQualityUCI.zip’ saved [1543989/1543989]

Archive:  AirQualityUCI.zip
  inflating: AirQualityUCI.csv       
  inflating: AirQualityUCI.xlsx      


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [3]:
target = 'NO2(GT)'
target

'NO2(GT)'

In [4]:
df.corr()[['NO2(GT)']]

,NO2(GT)
CO(GT),0.723154
PT08.S1(CO),0.284508
NMHC(GT),0.099541
C6H6(GT),0.402581
PT08.S2(NMHC),0.334108
NOx(GT),0.795888
PT08.S3(NOx),-0.440202
NO2(GT),1.000000
PT08.S4(NO2),0.010185
PT08.S5(O3),0.439057


In [5]:
df.head()
df = df[['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)','PT08.S2(NMHC)', 'NOx(GT)','NO2(GT)','PT08.S5(O3)']]

In [6]:
df.head()

,CO(GT),PT08.S1(CO),NMHC(GT),PT08.S2(NMHC),NOx(GT),NO2(GT),PT08.S5(O3)
0,2.6,1360.00,150,1045.50,166.0,113.0,1267.50
1,2.0,1292.25,112,954.75,103.0,92.0,972.25
2,2.2,1402.00,88,939.25,131.0,114.0,1074.00
3,2.2,1375.50,80,948.25,172.0,122.0,1203.25
4,1.6,1272.25,51,835.50,131.0,116.0,1110.00


In [7]:
X = df.drop(target,axis=1)
y = df[[target]]

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y)
value = y_test

In [9]:
print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)

(7017, 6)
(2340, 6)
(7017, 1)
(2340, 1)


In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.fit_transform(x_train)
scaler.fit(x_test)
x_test = scaler.fit_transform(x_test)
scaler.fit(y_train)
y_train = scaler.fit_transform(y_train)
scaler.fit(y_test)
y_test = scaler.fit_transform(y_test)

In [12]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
print (x_train.shape)
print (x_test.shape)

(7017, 6, 1)
(2340, 6, 1)


In [13]:
print (y_train.shape)
print (y_test.shape)

(7017, 1)
(2340, 1)


In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
def RNN_MODEL():
    # have to convert to 3D for feeding the data
    regressor = Sequential()
    regressor.add(LSTM(units = 50,return_sequences = True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50,return_sequences = True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units = 1))
    regressor.compile(optimizer = 'adam',loss = 'mean_squared_error',metrics=['accuracy'])
    return regressor

In [15]:
regressor = RNN_MODEL()

In [16]:
regressor.fit(x_train,y_train,epochs = 100)

Epoch 1/100
220/220 [==============================] - 1s 6ms/step - loss: 0.0262 - accuracy: 0.1753
Epoch 2/100
220/220 [==============================] - 1s 6ms/step - loss: 0.0216 - accuracy: 0.1751
Epoch 3/100
220/220 [==============================] - 1s 6ms/step - loss: 0.0192 - accuracy: 0.1751
Epoch 4/100
220/220 [==============================] - 1s 6ms/step - loss: 0.0179 - accuracy: 0.1753
Epoch 5/100
220/220 [==============================] - 1s 6ms/step - loss: 0.0169 - accuracy: 0.1753
Epoch 6/100
220/220 [==============================] - 1s 6ms/step - loss: 0.0161 - accuracy: 0.1750
Epoch 7/100
220/220 [==============================] - 1s 6ms/step - loss: 0.0157 - accuracy: 0.1751
Epoch 8/100
220/220 [==============================] - 1s 6ms/step - loss: 0.0153 - accuracy: 0.1753
Epoch 9/100
220/220 [==============================] - 1s 6ms/step - loss: 0.0149 - accuracy: 0.1751
Epoch 10/100
220/220 [==============================] - 1s 6ms/step - loss: 0.0146 - accura

In [17]:
regressor.evaluate(x_test,y_test)

74/74 [==============================] - 0s 3ms/step - loss: 0.0044 - accuracy: 0.1756


[0.004393612500280142, 0.1756410300731659]

In [18]:
y_pred = regressor.predict(x_test)

In [19]:
output = scaler.inverse_transform(y_pred)
real_output = []
for item in output:
    real_output.append((item[0]))

In [20]:
real_output

[23.857172,
 64.42667,
 112.770065,
 120.2482,
 65.70889,
 110.46687,
 63.412678,
 119.04587,
 87.03881,
 147.49277,
 -1.8923402,
 90.80992,
 -2.1717904,
 160.75705,
 77.12949,
 57.48177,
 79.80868,
 0.9491033,
 121.5107,
 128.97227,
 119.70108,
 118.30495,
 52.22721,
 17.310192,
 -1.3378589,
 117.69224,
 114.95016,
 111.584015,
 110.26712,
 124.21262,
 127.01504,
 123.85873,
 97.29119,
 123.22418,
 102.79485,
 65.24158,
 113.66189,
 157.92331,
 111.284035,
 -1.4234267,
 80.44533,
 110.93533,
 119.61113,
 104.42891,
 50.77041,
 151.60048,
 108.24125,
 0.6725959,
 37.474865,
 123.949554,
 165.03304,
 78.23787,
 -0.758426,
 63.335133,
 24.856796,
 121.33102,
 164.18358,
 131.86545,
 186.26053,
 155.31331,
 92.47761,
 234.64316,
 243.37892,
 48.732296,
 121.163635,
 124.94337,
 120.2964,
 99.07856,
 62.140182,
 112.19152,
 165.91368,
 96.139145,
 32.0095,
 86.356346,
 124.714966,
 -1.3201723,
 120.22998,
 103.22736,
 106.1008,
 199.90819,
 50.520103,
 -0.5845126,
 44.018314,
 122.00439,
 

In [21]:
value['predicted'] = np.array(real_output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
import matplotlib.pyplot as plt

In [23]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (100,100)

In [24]:
value.plot()